In [ ]:
#importing necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from os import walk
np.random.seed(1525)
#setting size of images to be sent into Hopfield
WE = 32
HE = 32
target_size=(WE,HE)

#Function to add noise to test images
def get_corrupted_input(input, corruption_level):
    corrupted = np.copy(input)
    inv = np.random.binomial(n=1, p=corruption_level, size=len(input))
    for i, v in enumerate(input):
        if inv[i]:
            corrupted[i] = 0 * v
    return corrupted

#Load our images from given file paths, resize and flatten them to use in Hopfield
def load_and_preprocess_images(image_paths, target_size):
    images = []
    for path in image_paths:
        image = Image.open(path)  # Open our images
        image = image.resize(target_size)
        image_array = np.array(image) / 255.0  # Normalize pixel values (0-1)
        images.append(image_array.flatten())  # Flatten the 2D array to 1D
    return images

#Loading training image paths all at once
res = []
fnames = None
for (dir_path, dir_names, file_names) in walk("/content/drive/MyDrive/train_imgs"):
  fnames = file_names

for name in fnames:
  res.append("/content/drive/MyDrive/train_imgs/" + name)
print("Images loaded for training : ", len(res))


#Loading test image paths and their relavant data like which category and how many examplars into a list
from os import walk
dir_main = "/content/drive/MyDrive/Gray2"
dir_path_list = []
for (dir_path, dir_names, file_names) in walk(dir_main):
    dir_path_list.append(dir_path)
dir_path_list.remove("/content/drive/MyDrive/Gray2")
dir_path_list.remove("/content/drive/MyDrive/Gray2/1exemplar")
dir_path_list.remove("/content/drive/MyDrive/Gray2/16exemplars")
dir_path_list.remove("/content/drive/MyDrive/Gray2/8exemplars")
dir_path_list.remove("/content/drive/MyDrive/Gray2/4exemplars")
dir_path_list.remove("/content/drive/MyDrive/Gray2/2exemplars")
test_img_info = []
dpl = "/content/drive/MyDrive/Gray2/1exemplar"
fnames = []
for (dir_path, dir_names, file_names) in walk(dpl):
  fnames = file_names
for file_names in fnames:
  test_img_info.append(['1exemplar', file_names, dpl +"/"+ file_names])
for dpl in dir_path_list:
  for (dir_path, dir_names, file_names) in walk(dpl):
    test_img_info.append([dir_path.split('/')[-2],dir_path.split('/')[-1], dir_path +"/"+ file_names[1]])
test_img_paths = []
for info in test_img_info:
  test_img_paths.append(info[2])
print("Images taken for testing : ", len(test_img_paths))



#Calling funtions to get train and test images
image_paths = res
vectorized_images = load_and_preprocess_images(image_paths, target_size)
random.shuffle(vectorized_images)
test_images = load_and_preprocess_images(test_img_paths, target_size)

In [ ]:
# Number of neurons in the network
num_neurons = WE * HE  # 32x32 images

# Number of patterns to store
num_patterns = len(vectorized_images)

# Size of each pattern (32x32 images)
pattern_size = WE * HE

# Generate random grayscale patterns for training
patterns = np.array(vectorized_images)

# Weight matrix initialization
weights = np.zeros((num_neurons, num_neurons))

# Hebbian learning rule to train the network
for pattern in patterns:
    pattern = pattern.reshape(num_neurons, 1)  # Reshape the pattern to a column vector
    weights += np.outer(pattern, pattern)

# Set diagonal elements to 0 (no self-connections)
np.fill_diagonal(weights, 0)

# Update function for continuous Hopfield network (asynchronous update)
def update_hopfield_network(input_pattern, weights, num_steps=100):
    pattern = input_pattern.copy()
    for _ in range(num_steps):
        neuron_idx = np.random.randint(0, num_neurons)  # Randomly select a neuron
        activation = np.dot(weights[neuron_idx], pattern)
        pattern[neuron_idx] = np.tanh(activation)  # Apply the activation function
    return pattern

# Function to calculate the similarity between two patterns
def calculate_similarity(pattern1, pattern2):
    return np.dot(pattern1, pattern2)

# Test the network by retrieving patterns
retrieved_patterns = []
similarities = []

patterns2 = np.array([get_corrupted_input(d, 0.1) for d in test_images])
p3 = np.array(test_images)

for i in range(patterns2.shape[0]):
    retrieved_pattern = update_hopfield_network(patterns2[i], weights)
    retrieved_patterns.append(retrieved_pattern)
    similarity = calculate_similarity(p3[i], retrieved_pattern)
    test_img_info[i].append(similarity * 100)
    similarities.append(similarity)

In [ ]:
#Storing results in an excel sheet
import pandas as pd
df = pd.DataFrame(test_img_info, columns=['Exemplar', 'Category', 'Path', 'Percent Correct'])
df.to_excel('/content/drive/MyDrive/crcT222.xlsx')